In [ ]:
pip install transformers

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM
import torch
import textwrap

In [ ]:
model_name = "felixoder/bug_detector_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
bug_fixer_model = "felixoder/bug_fixer_model"
fixer_tokenizer = AutoTokenizer.from_pretrained(bug_fixer_model)
fixer_model = AutoModelForCausalLM.from_pretrained(bug_fixer_model, torch_dtype=torch.float16, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/5.39k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.29M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/815 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
def classify_code(code):
    """Classify input code as 'buggy' or 'bug-free' using the trained model."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move model to the correct device

    inputs = tokenizer(code, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()

    return "bug-free" if predicted_label == 0 else "buggy"




In [ ]:
def fix_buggy_code(code):
    """Generate a fixed version of the buggy code using the bug fixer model."""
    prompt = f"### Fix this buggy Python code:\n{code} just give the fixed code nothing else\n### Fixed Python code:\n"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = fixer_tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = fixer_model.generate(
            **inputs,
            max_length=256,  # Reduce length for speed
            do_sample=False,  # Make it deterministic
            num_return_sequences=1  # Only one output
        )

    fixed_code = fixer_tokenizer.decode(outputs[0], skip_special_tokens=True)
    fixed_code = fixed_code.split("### Fixed Python code:")[1].strip() if "### Fixed Python code:" in fixed_code else fixed_code

    return textwrap.dedent(fixed_code).strip()


In [ ]:
# Example buggy code input
code_input = """
for in nge(0, 9
  print(i)
if val > 12:
  print("val {val} is greater")
else:
  print("val {val} is less")
"""

# Classify the code using the fine-tuned model
status = classify_code(code_input)

if status == "buggy":
    print("Buggy Code Detected")
    fixed_code = fix_buggy_code(code_input)
    print("Fixed Code:")
    print(fixed_code)
else:
    print("Bug-free Code")

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Buggy Code Detected
Fixed Code:
```python
for i in range(0, 9):
    print(i)
if val > 12:
    print("val {} is greater".format(val))
else:
    print("val {} is less".format(val))
```
